In [1]:
import argparse
import os
import sys
sys.path.append('../recommender/')
import numpy as np
from os import listdir, makedirs, remove
from collections import defaultdict
from os.path import isfile, join, exists
from cloth_detector import *
import gram_extractor as ge
import background_remover as br
from ntpath import basename
from tqdm import tqdm
from shutil import move

def get_files_from_path(image_path):
    return [join(image_path, f) for f in listdir(image_path) if isfile(join(image_path, f))]

def get_and_append(arr, filename):
    stored_arr = np.load(filename)
    new_arr = np.concatenate((stored_arr, arr), axis=0)
    np.save(filename, new_arr)

def get_grams_collection(cat):
    try:
        return np.load(get_grams_filename(cat))
    except:
        print("No files in the database!")
        
def get_filenames_collection(cat):
    try:
        return np.load(get_files_filename(cat))
    except:
        print("No files in the database!")

def get_grams_filename(cat):
    return f"../collections/{cat}_grams.npy"

def get_files_filename(cat):
    return f"../collections/{cat}_filenames.npy"

if __name__ == "__main__":
    print("Image Processing...")
#     ap = argparse.ArgumentParser()
#     ap.add_argument("-p", "--path", required=True, help="Path to the folder with images")
#     args = vars(ap.parse_args())
#     images_path = args["path"]
    images_path = "../../files"
    if exists(images_path):
        makedirs("../collections", exist_ok=True)
        makedirs("../static/image_data", exist_ok=True)
        temp = "../collections/temp.jpg"
        files = get_files_from_path(images_path)
        learner, id2cat, cat2id = load_learner()
        grams = defaultdict(list)
        filenames = defaultdict(list)
        for fn in tqdm(files):
            bboxes, cats, _ = predict(learner, fn)
            img = Image.open(fn)
            h, w = img.size
            bn = basename(fn)
            move(fn, f"../static/image_data/{bn}")
            for cat, bbox in zip(cats,bboxes):
                bbox = np.array([bbox[1]*h, bbox[0]*w, bbox[3]*h, bbox[2]*w])
                img.crop(tuple(bbox)).save(temp)
                br.remove_background(temp).save(temp)
                features_vector = ge.get_gram(temp)
                grams[cat].append(features_vector)
                filenames[cat].append(f"../static/image_data/{bn}")
            remove(temp)
        for cat in grams.keys():
            grams_path = get_grams_filename(cat)
            fns_path = get_files_filename(cat)
            if exists(grams_path) and exists(fns_path):
                get_and_append(grams[cat], grams_path)
                get_and_append(filenames[cat], fns_path)
                continue
            np.save(grams_path, grams[cat])
            np.save(fns_path, filenames[cat])
    else: print("No such directory. Make sure it exists")

Image Processing...


100%|██████████| 5/5 [01:08<00:00, 12.88s/it]


In [10]:
from IPython.display import FileLink, FileLinks
FileLink("../static/tmp/5a9901c9-6928-4ba1-9c12-2dbc9ccffe1a.jpg")

/home/vanio/fastai/old/Stylify/app/static/tmp/5a9901c9-6928-4ba1-9c12-2dbc9ccffe1a.jpg